 # 実験3（抽象構文木の利用）
 - 前回に続き抽象構文木に対する処理を学ぶ．
 - 例として後置記法（逆ポーランド記法）への変換や抽象構文木の描画を扱う．

 ## 抽象構文木の変換

 以下は前回と同じ（ライブラリの読み込みと10進整数を整数として切り出すパーサ`num`の定義）

In [0]:
from toyparsing import *

@parser_do
def num(run):
    return int(run(pat("[0-9]+")))

@parser_do
def factor(run):
    return run(num | w("(") >> expr << w(")"))

@parser_do
def term(run):
    a = run(factor)
    while True:
        r = run(pat("[*/]") & factor, nullable=True) 
        if r is None: break
        [op, b] = r
        a = [op, a, b]
    return a

@parser_do
def expr(run):
    a = run(term)
    while True:
        r = run(pat("[+-]") & term, nullable=True) 
        if r is None: break
        [op, b] = r
        a = [op, a, b]
    return a

 ## 抽象構文木から後置記法（逆ポーランド記法）への変換
 普通の数式は2項演算子をふたつの被演算子の中間に置いて`1+2`のように書く．これを**中置記法**と呼ぶ．
 それにたいして`+ 1 2`のように演算子を前に置く記法を**前置記法**（あるいはポーランド記法），
 `1 2 +`のように後に置く記法を**後置記法**（あるいは逆ポーランド記法）と呼ぶ．
 抽象構文木(AST)を後置記法の式になおすには，木を深さ優先探索し，葉から根に戻るときに演算子を最後に出力すればよい．
 以下のような非常に簡単なプログラムになる．

In [0]:
def postfix(ast):
    if isinstance(ast, int):
        return str(ast)
    [op, ast_left, ast_right] = ast         
    return postfix(ast_left) + " " + postfix(ast_right) + " " + op

In [0]:
@parser_do
def compile_to_postfix(run):
    return postfix(run(expr))

In [0]:
compile_to_postfix("1+2*3")    

In [0]:
compile_to_postfix("1-2-3-4")

In [0]:
compile_to_postfix("1-(2-(3-4))")

In [0]:
compile_to_postfix("1+2*3-4/5+7")

 ### [課題3]
 (1) 抽象構文木を前置記法の式（以下の例参照）に変換するプログラム`prefix`を作成せよ．

 (2) 抽象構文木をかっこを明示的に補った中置記法の式（以下の例参照）に変換するプログラム`infix`を作成せよ．

 [ヒント] どちらも式を後置記法になおす上記の`postfix`とほとんど同じになる．

In [0]:
@parser_do
def compile_to_prefix(run):
    return prefix(run(expr))

In [0]:
compile_to_prefix("1+2*3")  # + 1 * 2 3

In [0]:
compile_to_prefix("1-2-3-4")  # - - - 1 2 3 4

In [0]:
compile_to_prefix("1-(2-(3-4))")  # - 1 - 2 - 3 4

In [0]:
compile_to_prefix("1+2*3-4/5+7")  # + - + 1 * 2 3 / 4 5 7

In [0]:
@parser_do
def compile_to_infix(run):
    return infix(run(expr))

In [0]:
compile_to_infix("1+2*3")  # (1+(2*3))    

In [0]:
compile_to_infix("1-2-3-4")  # (((1-2)-3)-4)

In [0]:
compile_to_infix("1+2*3-4/5+7")  # (((1+(2*3))-(4/5))+7)

 ## Graphvizを用いた抽象構文木の描画
 Pythonから[Graphvis](https://www.graphviz.org/gallery/)を用いて抽象構文木を描画する．

 ライブラリの読み込み

In [0]:
from graphviz import Digraph

 ### graphvizの簡単な使い方
 有効グラフ（digraph)の土台(g)をつくり，それに頂点(node) n1, n2, n3と有向辺(edge)を追加していく．

In [0]:
g = Digraph()
g.node("n1", "1")
g.node("n2", "1")
g.node("n3", "+")
g.edge("n3","n1")
g.edge("n3","n2")

 最後にグラフの土台(g)を評価するとJupyterノートブックに直接描画される（ファイルに保存することも可能）
 詳しい使い方： [公式マニュアル](https://graphviz.readthedocs.io/en/stable/manual.html)

In [0]:
g

 ### [課題4]
 与えられた抽象構文木をGraphvizを用いて描画するプログラム`draw_ast`をつくれ．
 `draw_ast`は引数として抽象構文木`ast`と最初の頂点の名前を表す文字列`node`を取る．

 [ヒント] 重複の無い頂点の新しい名前を生成しながら処理を進めるのがポイント．
 そこで最初に与えられた頂点の名前の末尾に適当な文字列（例えば"L","R"）を接尾辞として付け足して
 新しい頂点の名前を作り出すとよい．

In [0]:
@parser_do
def draw_expr(run):
    global g # 次の代入ではグローバル変数`g`に代入
    g = Digraph()
    draw_ast(run(expr), "root")

In [0]:
draw_expr("1+2*3")
g

In [0]:
draw_expr("(1+2)*3")
g

In [0]:
draw_expr("1-2-3-4")
g

In [0]:
draw_expr("1-(2-(3-4))")
g